In [ ]:
import json

import pandas as pd
import numpy as np
import seaborn as sns
import requests
from pyspark.sql import Row

from matplotlib import pyplot as plt
from urllib.request import urlretrieve
from collections import defaultdict

## Gather data using API

[Note: The data was collected in Databricks and downloaded to a VM and then uploaded to Deepnote]


Data sources provide API Data retrieved in Databricks both as pandas dataftame and Spark dataframe convverted to CSV.
In the case of Spark files, the parquet files were coalesced into one file and saved into the CSV
SOURCE LINKS
- https://data.cnra.ca.gov/dataset/well-completion-reports
- https://data.ca.gov/dataset/household-water-supply-shortage-reporting-system-data
- https://data.cnra.ca.gov/dataset/periodic-groundwater-level-measurements

##### DATABRICKS AND DEEPNOTE
- The data was read using API calls from CNRA well water pages. The page links have an option for Data API with exmaple usage. The query strings were used in the request call in Python
- On an average about 5000 records were retrieved and the rest in chunks and placed into an extending list. At this point we have two options. We can convert the entire list to a dataframe and save as CSV in Databricks or Deepnote and use it OR the list can be converted, row by row into spark dataframe and saved as CSV.
- Pandas dataframe saved as CSV should be read back as Pandas DF.
- Spark dataframe saved as CSV should be read back as Spark dataframe or else you will encounter read error
- Dockerfile and init.py are set in Deepnote so that Spark can be used.

### Get Shortage Data

In [ ]:
api_retrieve = requests.get('https://data.cnra.ca.gov/api/3/action/datastore_search?resource_id=e1fd9f48-a613-4567-8042-3d2e064d77c8&limit=9000').json()
#Create a list of records
data_shortage = api_retrieve['result']['records']
pd_df_shortage = pd.DataFrame(data_shortage)

#Save as CSV if not present
pd_df_shortage.to_csv('/dbfs/FileStore/shortage.csv')



NameError: name 'requests' is not defined

In [ ]:
len(pd_df_shortage)

3774

### Get Well Completion Data

In [ ]:
#Create a list of records

well_completion_api = requests.get('https://data.cnra.ca.gov/api/3/action/datastore_search?resource_id=8da7b93b-4e69-495d-9caa-335691a1896b&limit=3200').json()
list_data_completion = well_completion_api['result']['records']

while well_completion_api['result']['records']:
    well_completion_api = requests.get('https://data.cnra.ca.gov'+well_completion_api['result']['_links']["next"]).json()
    list_data_completion.extend(well_completion_api['result']['records'])

pd_df_wellcompletion = pd.DataFrame(list_data_completion)

KernelInterrupted: Execution interrupted by the Jupyter kernel.

### 

### Get Station Data

In [ ]:
#Create a list of records

station_api = requests.get('https://data.cnra.ca.gov/api/3/action/datastore_search?resource_id=8da7b93b-4e69-495d-9caa-335691a1896b&limit=3200').json()
list_data_station = station_api['result']['records']

while station_api['result']['records']:
    station_api = requests.get('https://data.cnra.ca.gov'+station_api['result']['_links']["next"]).json()
    list_data_station.extend(station_api['result']['records'])

pd_df_station = pd.DataFrame(list_data_station)

KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
pd_df_station.sample(10)

### Get groundwater data

In [ ]:
#Create a list of records

groundwater_request_api = requests.get('https://data.cnra.ca.gov/api/3/action/datastore_search?resource_id=bfa9f262-24a1-45bd-8dc8-138bc8107266&limit=4000').json()
list_data_groundwater = groundwater_request_api['result']['records']
while groundwater_request_api['result']['records']:
    groundwater_request_api = requests.get('https://data.cnra.ca.gov'+groundwater_request_api['result']['_links']["next"]).json()
    list_data_groundwater.extend(groundwater_request_api['result']['records'])
    
df_groundwater = spark.createDataFrame(Row(**row) for row in list_data_groundwater)
df_groundwater.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/WaterWell/groundwater.csv")    

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b042e2da-6536-449d-95b8-d85fa08825de' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>